# Instructor Do: Intro to VADER Sentiment

In [4]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv("../../../../../../.env")


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jeffr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")
type(api_key)


str

In [13]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [8]:
# Fetch all the news about Facebook Libra
diem_headlines = newsapi.get_everything(
    q="facebook AND diem",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Diem: {diem_headlines['totalResults']}")

# Show sample article
diem_headlines["articles"][0]



Total articles about Facebook Diem: 125


{'source': {'id': None, 'name': 'Slashdot.org'},
 'author': 'msmash',
 'title': "Facebook's David Marcus, Creator of Embattled Diem Project, To Leave Company",
 'description': 'David Marcus, one of the top executives at Meta Platforms and the co-creator of the yet-to-be-launched Diem digital currency, is leaving the company after seven years to pursue other projects. From a report: Marcus, who joined the Facebook parent company in 2…',
 'url': 'https://tech.slashdot.org/story/21/11/30/1753219/facebooks-david-marcus-creator-of-embattled-diem-project-to-leave-company',
 'urlToImage': 'https://a.fsdn.com/sd/topics/facebook_64.png',
 'publishedAt': '2021-11-30T17:53:00Z',
 'content': "Marcus, who joined the Facebook parent company in 2014 from PayPal Holdings, ran the Messenger service for years before moving over to form the company's blockchain division in 2018. He spent the las… [+797 chars]"}

In [10]:
# Create the Facebook Libra sentiment scores DataFrame
diem_sentiments = []

for article in diem_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        diem_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
diem_df = pd.DataFrame(diem_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
diem_df = diem_df[cols]

diem_df.head()



,date,text,compound,positive,negative,neutral
0,2021-11-30,"Marcus, who joined the Facebook parent company...",0.0000,0.000,0.000,1.000
1,2021-12-09,Dec 9 (Reuters) - Meta Platforms Inc's (FB.O) ...,0.2263,0.058,0.000,0.942
2,2021-12-02,Facebook Meta signJustin Sullivan/Getty Images...,0.6486,0.159,0.000,0.841
3,2021-12-09,"Meta Platforms Incs cryptocurrency wallet, Nov...",0.0772,0.091,0.054,0.855
4,2021-11-30,"David Marcus, the head of Facebook's cryptocur...",-0.5106,0.000,0.088,0.912


In [11]:
# Get descriptive stats from the DataFrame
diem_df.describe()



,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.129796,0.059550,0.027200,0.913270
std,0.359838,0.061759,0.061244,0.083334
min,-0.921700,0.000000,0.000000,0.567000
25%,0.000000,0.000000,0.000000,0.876000
50%,0.115350,0.051000,0.000000,0.927000
75%,0.381800,0.088750,0.037250,0.976750
max,0.859200,0.264000,0.329000,1.000000
